In [1]:
!pip install imblearn
!pip install pydrive
!pip install mlxtend

    100% |████████████████████████████████| 174kB 10.3MB/s 
    100% |████████████████████████████████| 5.4MB 6.4MB/s 
  Found existing installation: scikit-learn 0.19.2
    Uninstalling scikit-learn-0.19.2:
      Successfully uninstalled scikit-learn-0.19.2
    100% |████████████████████████████████| 993kB 20.7MB/s 
  Running setup.py bdist_wheel for pydrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive
    100% |████████████████████████████████| 1.3MB 15.2MB/s 


In [0]:
#Accessing Google drive File
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [49]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
#Import Libraries
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
#Store Model to Local Disk
from sklearn.externals import joblib
#confusion matrix
from mlxtend.evaluate import confusion_matrix 
from mlxtend.plotting import plot_confusion_matrix

%matplotlib inline

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#Get the file
#Replace the id with id of file you want to access
normal_1 = drive.CreateFile({'id':'13AR0sS1pndF0fcsssxmdjQRv_1Bv5aBNpkT'}) 
normal_1.GetContentFile('normal_1.csv')

normal_2 = drive.CreateFile({'id':'1Z0DO8M1Qco07kyVoxooogxXBx6XYGBzJd'}) 
normal_2.GetContentFile('normal_2.csv')

abnormal = drive.CreateFile({'id':'12zFHDXVcccccorRrHHhYrA1n82VQLuawsl'}) 
abnormal.GetContentFile('abnormal.csv')

In [0]:
# Read file as panda dataframe
normal_1 = pd.read_csv('normal_1.csv', header=None) 
normal_2 = pd.read_csv('normal_2.csv', header=None)
abnormal = pd.read_csv('abnormal.csv', header=None)

In [0]:
# Read some rows for smaller computations
normal_1 = pd.read_csv('normal_1.csv', skiprows = np.arange(100, normal_1.shape[0])) 
normal_2 = pd.read_csv('normal_2.csv', skiprows = np.arange(100, normal_2.shape[0])) 
abnormal = pd.read_csv('abnormal.csv', skiprows = np.arange(50, abnormal.shape[0])) 

In [0]:
#Merge Normal 2 and abnormal
anomaly = pd.DataFrame(np.concatenate([normal_1.values, abnormal.values]))

In [0]:
#Get train data
X_train=normal_1.iloc[:, 1:normal_1.shape[1]].values
y_train = np.repeat(0, len(X_train))

X_test=anomaly.iloc[:, 1:anomaly.shape[1]].values
y_test = np.concatenate([np.repeat(0, len(normal_2)), np.repeat(1, len(abnormal))])

In [0]:
# Apply the random under-sampling for test data
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(return_indices=True, random_state = 42)
X_test, y_test, idx_resampled = rus.fit_sample(X_test, y_test)

In [0]:
#Normalize the train dataset
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)

#Normalize the test dataset
X_test = min_max_scaler.transform(X_test)

In [37]:
#Apply One Class SVM
from sklearn import svm
clf = svm.OneClassSVM(kernel = 'linear',
                      nu=(len(y_test[y_test==1])/X_train.shape[0]))  
clf.fit(X_train)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto_deprecated',
      kernel='linear', max_iter=-1, nu=0.49, random_state=None,
      shrinking=True, tol=0.001, verbose=False)

In [0]:
#Predict on test data
y_pred=clf.predict(X_test)

y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1

In [39]:
#Calculate accuracy
from sklearn import metrics
print("Accuracy %0.2f" % (100*metrics.accuracy_score(y_test, y_pred)))

Accuracy 26.53


In [40]:
# save the model to disk
filename = 'model.sav'
joblib.dump(clf, filename)

['model.sav']

In [0]:
model_file = drive.CreateFile({'title' : 'model.sav'})
model_file.SetContentFile('model.sav')
model_file.Upload()